# Overview 🚀

This notebook demonstrates how to serve a trained image classification model using FastAPI and Jinja2.

## Workflow

1. **Prediction Pipeline:** 🤖 Deploy our trained model for inference.
2. **FastAPI Setup:**  
    - 🔗 Define routes for training and prediction.
    - 🧩 Integrate Jinja2 for dynamic HTML rendering.
    - 📄 Provide interactive API documentation.
3. **User Interface:**  
    - 🔌 Access REST API endpoints for programmatic interaction.
    - 🌐 Use a custom website for user-friendly image classification.

This approach enables both API access and a web interface for efficient image classification tasks.

## Jinja ✨

Jinja is a modern and designer-friendly templating engine for Python, used to generate dynamic HTML content by combining templates with data.

**Why use Jinja with FastAPI?**
- **Separation of concerns:** Keeps logic and presentation separate.
- **Dynamic rendering:** Allows HTML pages to display backend data.
- **Maintainability:** UI updates do not require backend changes.

**Typical workflow:**
1. Define HTML templates in a `templates` directory.
2. Use `Jinja2Templates` in FastAPI to load and render templates.
3. Pass data from backend to template for dynamic content.

This approach helps organize code and simplifies updates to the user interface.

## FastAPI 🚀

FastAPI 🚀 is a modern, fast (high-performance) web framework for building APIs with Python 3.7+ based on standard Python type hints. It is designed for ease of use, speed, and automatic generation of interactive API documentation.

**Key features:**
- ⚡ **Speed:** Built on Starlette and Pydantic, FastAPI is one of the fastest Python frameworks.
- 📝 **Type hints:** Uses Python type hints for data validation and editor support.
- 📄 **Automatic docs:** Generates interactive API documentation (Swagger UI and ReDoc) automatically.
- 🔄 **Asynchronous support:** Handles async programming for high concurrency.
- 🔌 **Easy integration:** Works well with data validation, authentication, and templating engines like Jinja2.

FastAPI is ideal for building RESTful APIs, microservices, and backend services efficiently.

In [1]:
# Import necessary FastAPI modules and other dependencies
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.templating import Jinja2Templates
import os
from ImageClassification.utils import decodeImage
# Our pipeline.
from ImageClassification.pipeline.predict import DogCat

# Set environment variables for language and locale
os.putenv('LANG', 'en_US.UTF-8')
os.putenv('LC_ALL', 'en_US.UTF-8')

# Initialize FastAPI app
app = FastAPI(docs_url="/docs", redoc_url="/redoc")

# Add CORS middleware to allow cross-origin requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],         # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],         # Allow all HTTP methods
    allow_headers=["*"],         # Allow all headers
)

# Set up Jinja2 templates directory
templates = Jinja2Templates(directory="templates")

# ClientApp class to manage image filename and classifier instance
class ClientApp:
    def __init__(self):
        self.filename = "inputImage.jpg"         # Default filename for input image
        self.classifier = DogCat(self.filename)  # Initialize classifier with filename

# Create a global instance of ClientApp
clApp = ClientApp()

# Route for home page, renders index.html template
@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

# Route to trigger training by running main.py script
@app.post("/train")
async def trainRoute():
    os.system("python main.py")  # Execute training script
    return {"message": "Training done successfully!"}

# Route to handle image prediction requests
@app.post("/predict")
async def predictRoute(request: Request):
    data = await request.json()                  # Parse JSON data from request
    image = data['image']                        # Get image data
    decodeImage(image, clApp.filename)           # Decode and save image to file
    result = clApp.classifier.predictiondogcat() # Run prediction on saved image
    return JSONResponse(content=result)          # Return prediction result as JSON

2025-08-18 21:40:56.906837: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 21:40:56.907065: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-18 21:40:56.944780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-18 21:40:57.952407: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Execute the following command then:

* 🚀 Visit [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs) to view Swagger UI and manually trigger the training pipeline using REST APIs.
* 🖼️ Visit [http://127.0.0.1:8000](http://127.0.0.1:8000) to view the HTML website for prediction.

**Note:** 🔒 This is only hosted on your device and not exposed to the public.

In [2]:
# Run the FastAPI app using Uvicorn with 2 worker processes, accessible at port 8000
!uvicorn app:app --port 8000 --workers 2

ERROR:    [Errno 98] Address already in use
